In [1]:
import numpy as np
import pandas as pd
import copy

In [2]:
file = 'C:/Users/0630r/inha2/Research/Dart/Dart_file/support_data/dart-2020-2022.xlsx'
df = pd.read_excel(file, sheet_name = [0], header = 2)
df = df[0]
# OverView sheet에서 필요한 정보만 뽑아오기 
df = df[['symbol', 'entity_name', 'business_rid', 'address_road_name_x', 'industry_id_x', 'market', '종업원수']]
df.head()

C:\Users\0630r\AppData\Local\Temp\ipykernel_3564\2974668659.py:2: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df = pd.read_excel(file, sheet_name = [0], header = 2)


,symbol,entity_name,business_rid,address_road_name_x,industry_id_x,market,종업원수
0,KRX:000020,동화약품,1.108100e+09,"서울 중구 남대문로9길 24, 6층-8층",KRI:10C2121000,KOSPI,756.0
1,KRX:000040,KR모터스,3.058100e+09,경남 창원시 성산구 완암로 28,KRI:10C3192000,KOSPI,52.0
2,KRX:000050,경방,1.078105e+09,서울 영등포구 영중로 15,KRI:10G4719000,KOSPI,191.0
3,KRX:000070,삼양홀딩스,2.088101e+09,서울 종로구 종로33길 31,KRI:10K6499200,KOSPI,535.0
4,KRX:000080,하이트진로,2.148101e+09,서울 강남구 영동대로 714,KRI:10C1112200,KOSPI,3077.0


In [3]:
# parquet으로 저장
df.to_parquet('orign_overview.pqt')

## 데이턴 전처리

In [2]:
path = 'C:/Users/0630r/inha2/Research/Dart/Dart_file/real_code/pqt_file/'
orign_overview = pd.read_parquet(path + 'orign_overview.pqt', engine = 'pyarrow')

In [3]:
orign_overview.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2517 entries, 0 to 2516
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   symbol               2517 non-null   object 
 1   entity_name          2517 non-null   object 
 2   business_rid         2496 non-null   float64
 3   address_road_name_x  2489 non-null   object 
 4   industry_id_x        2517 non-null   object 
 5   market               2517 non-null   object 
 6   종업원수                 2422 non-null   float64
dtypes: float64(2), object(5)
memory usage: 137.8+ KB


In [4]:
orign_overview.rename(columns = {'entity_name': '업체명', 'business_rid': '사업자등록번호', 'address_road_name_x': '도로명주소', 'industry_id_x': '업종분류코드'}, inplace = True) # 열이름 지정
orign_overview['도로명주소'].fillna('None', inplace = True) # 도로명주소 없는 애들 None으로 저장
orign_overview['종업원수'].fillna(0, inplace = True) # 종업원 수 null 값인 애들 0으로 대체

In [5]:
## 주소를 시도로 구분, 업종분류를 한국표준산업분류로 구분
orign_overview['도로명주소'] = np.where(orign_overview['도로명주소'] != None, orign_overview['도로명주소'].apply(lambda x:x[:2]), orign_overview['도로명주소']) # 주소를 시도로 구분
orign_overview['업종분류코드'] = orign_overview['업종분류코드'].apply(lambda x:x[6]) ## nunique = 17개
orign_overview.drop(orign_overview[orign_overview['업종분류코드'] == 'K'].index, axis = 0, inplace = True) #금융업 업종 삭제 
orign_overview.dropna(subset=['사업자등록번호'], inplace = True) # 사업자등록번호가 없는 데이터 삭제 
orign_overview.reset_index(drop=True, inplace = True)
orign_overview.info()

## 종업원수가 년도별로 되어 있지 않다. 이후 employee data를 통해 merge 진행.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2350 entries, 0 to 2349
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   symbol   2350 non-null   object 
 1   업체명      2350 non-null   object 
 2   사업자등록번호  2350 non-null   float64
 3   도로명주소    2350 non-null   object 
 4   업종분류코드   2350 non-null   object 
 5   market   2350 non-null   object 
 6   종업원수     2350 non-null   float64
dtypes: float64(2), object(5)
memory usage: 128.6+ KB


In [6]:
orign_overview 
# 2350개

,symbol,업체명,사업자등록번호,도로명주소,업종분류코드,market,종업원수
0,KRX:000020,동화약품,1.108100e+09,서울,C,KOSPI,756.0
1,KRX:000040,KR모터스,3.058100e+09,경남,C,KOSPI,52.0
2,KRX:000050,경방,1.078105e+09,서울,G,KOSPI,191.0
3,KRX:000080,하이트진로,2.148101e+09,서울,C,KOSPI,3077.0
4,KRX:000100,유한양행,1.188101e+09,서울,C,KOSPI,1938.0
...,...,...,...,...,...,...,...
2345,KRX:448730,삼성FN리츠,5.478703e+09,서울,L,KOSPI,0.0
2346,KRX:448780,마이크로엔엑스,5.038151e+09,대구,C,KONEX,70.0
2347,KRX:451800,한화리츠,2.888802e+09,서울,L,KOSPI,0.0
2348,KRX:456190,큐라켐,1.378195e+09,충북,C,KONEX,29.0


In [12]:
## 업종분류코드 
## A: 농업, 임업 및 어업
## B: 광업
## C: 제조업
## D: 전기, 가스, 증기 및 공기 조절 공급업
## E: 수도, 하수 및 폐기물 처리, 원료 재생업
## F: 건설업
## G: 도매 및 소매업
## H: 운수 및 창고업
## I: 숙박 및 음식점업
## J: 정보통신
## K: 금융 및 보험업
## L: 부동산업
## M: 전문, 과학 및 기술 서비스업
## N: 사업시설 관리, 사업 지원 및 임대 서비스업
## O: 공공 행정, 국방 및 사회보장 행
## P: 교육 서비스업
## Q: 보건업 및 사회복지 서비스업
## R: 예술, 스포츠 및 여가관련 서비스업
## S: 협회 및 단체, 수리 및 기타 개인 서비스업
## T: 가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동
## U: 국제 및 외국기관

In [13]:
orign_overview.to_parquet('overView_final.pqt')
# 완

In [14]:
# orign_overview, overView_final 두개 저장